<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9 

In [ ]:
!ls -tl

total 4
drwxr-xr-x 1 root root 4096 Jan 20 17:27 sample_data


In [ ]:
!ls -tl sample_data

total 55504
-rw-r--r-- 1 root root 18289443 Jan 20 17:27 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jan 20 17:27 mnist_train_small.csv
-rw-r--r-- 1 root root   301141 Jan 20 17:27 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jan 20 17:27 california_housing_train.csv
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


**Spark Session:** SparkSession은 Spark 2.0부터 엔트리 포인트로 사용된다. 그 이전에는 SparkContext가 사용되었다. SparkSession을 이용해 RDD, 데이터 프레임등을 만든다. SparkSession은 SparkSession.builder를 호출하여 생성하며 다양한 함수들을 통해 세부 설정이 가능하다

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

In [ ]:
spark

**Python 객체를 RDD로 변환해보기**

**1> Python 리스트 생성**

In [ ]:
name_list_json = [ '{"name": "keeyong"}', '{"name": "benjamin"}', '{"name": "claire"}' ]

In [ ]:
for n in name_list_json:
  print(n)

{"name": "keeyong"}
{"name": "benjamin"}
{"name": "claire"}


In [ ]:
import json

for n in name_list_json:
  jn = json.loads(n)
  print(jn["name"])

keeyong
benjamin
claire


**2> 파이썬 리스트를 RDD로 변환. RDD로 변환되는 순간 Spark 클러스터의 서버들에 데이터가 나눠 저장됨 (파티션). 또한 Lazy Execution이 된다는 점 기억**

In [ ]:
rdd = spark.sparkContext.parallelize(name_list_json)

In [ ]:
rdd

ParallelCollectionRDD[4] at readRDDFromFile at PythonRDD.scala:262

In [ ]:
rdd.count()

3

In [ ]:
parsed_rdd = rdd.map(lambda el:json.loads(el))

In [ ]:
parsed_rdd

PythonRDD[6] at RDD at PythonRDD.scala:53

In [ ]:
parsed_rdd.collect()

[{'name': 'keeyong'}, {'name': 'benjamin'}, {'name': 'claire'}]

In [ ]:
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [ ]:
parsed_name_rdd.collect()

['keeyong', 'benjamin', 'claire']

**파이썬 리스트를 데이터프레임으로 변환하기**

In [ ]:
from pyspark.sql.types import StringType

df = spark.createDataFrame(name_list_json, StringType())

In [ ]:
df.count()

3

In [ ]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
df.select('*').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [ ]:
df.select('value').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [ ]:
from pyspark.sql import Row

row = Row("name") # Or some other column name
df_name = parsed_name_rdd.map(row).toDF()

In [ ]:
df_name.printSchema()

root
 |-- name: string (nullable = true)



In [ ]:
df_name.select('name').collect()

[Row(name='keeyong'), Row(name='benjamin'), Row(name='claire')]